In [16]:
import re, string,os
from glob import glob as gb
import pandas as pd
from tqdm import tqdm
from collections import Counter
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
from functions import *
from gensim.models import KeyedVectors
from scipy import spatial

plotting.style_('tab20',16)
base_path = "/media/ruben/OSDisk/Users/ruben.ros/Documents/GitHub/ParlaMintCase"

In [2]:
# Measuring Distance between Speech Elements, look if they are closer as the crisis progresses
# 1. 
#

In [7]:
data = data_loader.full('pl',"lemmatized")
data['date'] = [utils.find_date(x) for x in data['id']]
data = utils.add_metadata(data,'pl')
data = data[data['date'].str.contains('2019|2020')].reset_index(drop=True)

In [19]:
model = KeyedVectors.load(base_path + "/data/embeddings/word2vec_100_3_polish.bin")
vocab = set(model.wv.vocab)

def avg_(text):
    return np.array([model[w] for w in str(text).split(' ') if w in vocab]).mean(axis=0)

def cos_(e1,e2):
    return 1 - spatial.distance.cosine(e1, e2)

In [32]:
d = []

for date in tqdm(list(set(data['date']))):
    sentences = list(data[data['date'] == date]['text'])
    embeddings = [avg_(t) for t in sentences]
    sims = []
    for c,e in enumerate(embeddings):

        for c1,e1 in enumerate(embeddings):
            if c != c1:
                sims.append(cos_(e,e1))
    d.append([date,sum(sims) / len(sims)])

 16%|█▌        | 20/125 [06:42<35:15, 20.15s/it]  


KeyboardInterrupt: 

In [33]:
d

[['2020-03-02', 0.5471150715552651],
 ['2020-01-10', 0.5473529223605795],
 ['2019-03-14', nan],
 ['2019-03-21', nan],
 ['2019-01-31', 0.6416487066306052],
 ['2020-08-17', 0.5605624337060533],
 ['2019-02-22', 0.6151653649318806],
 ['2019-07-10', nan],
 ['2019-09-25', nan],
 ['2020-05-05', nan],
 ['2020-01-22', nan],
 ['2019-02-01', nan],
 ['2019-06-13', nan],
 ['2020-07-23', nan],
 ['2020-06-01', 0.6190040628485443],
 ['2020-03-27', nan],
 ['2020-08-14', nan],
 ['2019-12-12', nan],
 ['2020-06-02', 0.6065921030461957],
 ['2019-05-16', nan]]

0.6129808978616351